In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import lightgbm as gbm

In [2]:
#read dataset

data = pd.read_csv("/home/jagadeesh/datasets/diabetes.csv")

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
trn_x, val_x, trn_y, val_y = train_test_split(data.drop('Outcome',axis=1).values,data.Outcome.values,test_size=0.10)

In [5]:
trn_dataset = gbm.Dataset(trn_x,trn_y)
val_dataset = gbm.Dataset(trn_x,trn_y)


clf = gbm.LGBMClassifier()

In [6]:
param_grid = {
    'objective': ['binary'],
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40],
    'learning_rate': [0.20,0.25,0.30,0.40,0.50],
    'max_depth': [6,8,10,12,14,16],
    'min_data_in_leaf': [5,10,15],
    'max_bin' : [50,100,150],
    'num_leaves' : [128,256,360]
}

gs = GridSearchCV(clf, param_grid, cv=3)

In [7]:
gs.fit(trn_x,trn_y)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'objective': ['binary'], 'learning_rate': [0.2, 0.25, 0.3, 0.4, 0.5], 'n_estimators': [20, 40], 'max_depth': [6, 8, 10, 12, 14, 16], 'min_data_in_leaf': [5, 10, 15], 'max_bin': [50, 100, 150], 'num_leaves': [128, 256, 360]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [8]:
gs.best_params_

{'learning_rate': 0.2,
 'max_bin': 50,
 'max_depth': 10,
 'min_data_in_leaf': 15,
 'n_estimators': 20,
 'num_leaves': 128,
 'objective': 'binary'}

In [9]:
gs.best_score_

0.7670043415340086

In [10]:
from sklearn.metrics import accuracy_score

rounder = lambda x: 1 if x > 0.51 else 0

predicted = []

for p in gs.best_estimator_.predict(val_x):
    predicted.append(rounder(p))
    
accuracy_score(val_y, predicted)   

0.7402597402597403